In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/trainset.zip'

Archive:  /content/drive/MyDrive/trainset.zip
   creating: trainset/
   creating: trainset/data/
  inflating: trainset/data/signal0.txt  
  inflating: trainset/data/signal0_1.txt  
  inflating: trainset/data/signal1.txt  
  inflating: trainset/data/signal10.txt  
  inflating: trainset/data/signal100.txt  
  inflating: trainset/data/signal100_1.txt  
  inflating: trainset/data/signal101.txt  
  inflating: trainset/data/signal101_1.txt  
  inflating: trainset/data/signal102.txt  
  inflating: trainset/data/signal102_1.txt  
  inflating: trainset/data/signal103.txt  
  inflating: trainset/data/signal103_1.txt  
  inflating: trainset/data/signal104.txt  
  inflating: trainset/data/signal104_1.txt  
  inflating: trainset/data/signal105.txt  
  inflating: trainset/data/signal105_1.txt  
  inflating: trainset/data/signal106.txt  
  inflating: trainset/data/signal106_1.txt  
  inflating: trainset/data/signal107.txt  
  inflating: trainset/data/signal107_1.txt  
  inflating: trainset/data/signa

In [ ]:
import os

input_dir = '/content/trainset/data'
target_dir = '/content/trainset/labels'

input_data_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".txt")
    ]
)
target_label_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".txt") and not fname.startswith(".")
    ]
)
 
print("Number of samples:", len(input_data_paths))
print(len(target_label_paths))
 
for input_path, target_path in zip(input_data_paths[:10], target_label_paths[:10]):
    print(input_path, "|", target_path)

Number of samples: 1000
1000
/content/trainset/data/signal0.txt | /content/trainset/labels/label0.txt
/content/trainset/data/signal0_1.txt | /content/trainset/labels/label0_1.txt
/content/trainset/data/signal1.txt | /content/trainset/labels/label1.txt
/content/trainset/data/signal10.txt | /content/trainset/labels/label10.txt
/content/trainset/data/signal100.txt | /content/trainset/labels/label100.txt
/content/trainset/data/signal100_1.txt | /content/trainset/labels/label100_1.txt
/content/trainset/data/signal101.txt | /content/trainset/labels/label101.txt
/content/trainset/data/signal101_1.txt | /content/trainset/labels/label101_1.txt
/content/trainset/data/signal102.txt | /content/trainset/labels/label102.txt
/content/trainset/data/signal102_1.txt | /content/trainset/labels/label102_1.txt


In [ ]:
import numpy as np
import random
random.Random(1337).shuffle(input_data_paths)
random.Random(1337).shuffle(target_label_paths)
X = np.zeros((len(input_data_paths), 10))
y = np.zeros((len(input_data_paths), 1))
for i in range(len(input_data_paths)):
    X[i] = np.loadtxt(input_data_paths[i])
    y[i] = np.loadtxt(target_label_paths[i])

In [ ]:
X = X.reshape((len(input_data_paths), 10, 1))
y = y.reshape((len(input_data_paths), 1))
Y = np.zeros((len(input_data_paths), 1))
Y[y == 0] = 1
Y = np.hstack((Y,y))

In [ ]:
print(y[0])
print(Y[0])
print(X[1])
print(y[1])
print(Y.shape)

[0.]
[1. 0.]
[[-2.38154724]
 [-2.44343662]
 [-2.49823317]
 [-2.54416726]
 [-2.58012471]
 [-2.60573105]
 [-2.62135232]
 [-2.6280174 ]
 [-2.62727467]
 [-2.62100217]]
[0.]
(1000, 2)


In [ ]:
X_train = X[:-100]
X_test = X[-100:]
y_train = Y[:-100]
y_test = Y[-100:]

In [ ]:
print(X_train[-1])
print(X_test[0])

[[ 0.0779558 ]
 [ 0.05769044]
 [ 0.00363233]
 [-0.08257754]
 [-0.19761563]
 [-0.33681406]
 [-0.49455586]
 [-0.66469091]
 [-0.84093843]
 [-1.0172477 ]]
[[ 0.1559116 ]
 [ 0.11538087]
 [ 0.00726466]
 [-0.16515507]
 [-0.39523125]
 [-0.67362813]
 [-0.98911172]
 [-1.32938183]
 [-1.68187685]
 [-2.0344954 ]]


In [54]:
# 導入函式庫
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam
# 建立簡單的線性執行的模型
model = Sequential()
# 加 RNN 隱藏層(hidden layer)
model.add(SimpleRNN(
    # 如果後端使用tensorflow，batch_input_shape 的 batch_size 需設為 None.
    # 否則執行 model.evaluate() 會有錯誤產生.
    batch_input_shape=(None, 10, 1), 
    units= 10,
    unroll=True
)) 
# 加 output 層
model.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))
callbacks = [
    keras.callbacks.ModelCheckpoint("CrestDetectRnn.h5", save_best_only=True)
]
# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# 一批訓練多少張圖片     
BATCH_INDEX = 0
BATCH_SIZE = 50     
# 訓練模型 4001 次
for step in range(1, 100001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    randnum = np.random.randint(-3, 3)
    X_batch = X_batch + randnum
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    # 逐批訓練
    loss = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    if BATCH_INDEX >= X_train.shape[0]: 
        BATCH_INDEX = 0 
    # 每 500 批，顯示測試的準確率
    if step % 500 == 0:
        # 模型評估
        randnum = np.random.randint(-3, 3)
        X_testbatch = X_test + randnum
        loss, accuracy = model.evaluate(X_testbatch, y_test, batch_size=y_test.shape[0], 
            verbose=False, callbacks = callbacks)
        print("test loss: {}  test accuracy: {}".format(loss,accuracy))
        

test loss: 0.5346643328666687  test accuracy: 0.7099999785423279
test loss: 0.3423246741294861  test accuracy: 0.7900000214576721
test loss: 0.22980248928070068  test accuracy: 0.8999999761581421
test loss: 0.30037686228752136  test accuracy: 0.8399999737739563
test loss: 0.24005725979804993  test accuracy: 0.8999999761581421
test loss: 0.2327681928873062  test accuracy: 0.9200000166893005
test loss: 0.4456665813922882  test accuracy: 0.7699999809265137
test loss: 0.35109958052635193  test accuracy: 0.7900000214576721
test loss: 0.21518102288246155  test accuracy: 0.9200000166893005
test loss: 0.4225171208381653  test accuracy: 0.7900000214576721
test loss: 0.3440425992012024  test accuracy: 0.8199999928474426
test loss: 0.2208857387304306  test accuracy: 0.8899999856948853
test loss: 0.24616986513137817  test accuracy: 0.9100000262260437
test loss: 0.21502207219600677  test accuracy: 0.8999999761581421
test loss: 0.5672076940536499  test accuracy: 0.7400000095367432
test loss: 0.22652

In [ ]:
#np.random.seed(1337)
np.random.randint(-4, 5)

2

In [58]:
# 預測(prediction)
X = X_test[10:20,:]
X = X-1
print(X)
predictions = np.argmax(model.predict(X), axis=-1)
# get prediction result
print(predictions)
print(y_test[10:20,1])

[[[ 1.95526779e+01]
  [ 1.90699850e+01]
  [ 1.83106461e+01]
  [ 1.73273976e+01]
  [ 1.61773964e+01]
  [ 1.49181491e+01]
  [ 1.36040138e+01]
  [ 1.22835241e+01]
  [ 1.09976577e+01]
  [ 9.77903980e+00]]

 [[-1.04067511e+01]
  [-1.06534606e+01]
  [-1.08983447e+01]
  [-1.11384826e+01]
  [-1.13713918e+01]
  [-1.15951952e+01]
  [-1.18086968e+01]
  [-1.20113464e+01]
  [-1.22030962e+01]
  [-1.23841739e+01]]

 [[ 1.47189751e-01]
  [ 1.78230388e+00]
  [ 3.38619935e+00]
  [ 4.91560254e+00]
  [ 6.33648348e+00]
  [ 7.62615809e+00]
  [ 8.77401659e+00]
  [ 9.78082522e+00]
  [ 1.06567516e+01]
  [ 1.14184441e+01]]

 [[ 5.85852917e+00]
  [ 6.55004329e+00]
  [ 7.07523696e+00]
  [ 7.41781180e+00]
  [ 7.57369327e+00]
  [ 7.55086735e+00]
  [ 7.36815559e+00]
  [ 7.05309512e+00]
  [ 6.63917341e+00]
  [ 6.16271862e+00]]

 [[ 1.90908586e-01]
  [ 1.86240883e-01]
  [ 1.67889426e-01]
  [ 1.32321715e-01]
  [ 7.74835200e-02]
  [ 2.95447806e-03]
  [-9.00304561e-02]
  [-1.98705840e-01]
  [-3.19042742e-01]
  [-4.461361

In [59]:
model.save('/content/drive/MyDrive/CrestDetectRnn_Best.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/CrestDetectRnn.h5')

In [ ]:
# 預測(prediction)
X = X_test[31:40,:]
predictions = np.argmax(model.predict(X), axis=-1)
# get prediction result
print(predictions)
print(y_test[31:40,1])

[1 0 1 0 1 1 1 1 1]
[1. 0. 1. 0. 1. 0. 1. 1. 1.]


In [ ]:
import tensorflow as tf
import keras
tf.__version__
keras.__version__

'2.4.3'